# The goal of this is to set up scheduled queries to MySQL, see if the data shows anything weird, then notify users if data is weird

## This version needs the credentials for all user and passwords for the stated services 

In [1]:
# send emails based on smtplib with information
def send_email(em_user='xxx', em_pwd='xxx', recipients=['test@example.com'], subject='attention', body='attention'):
    '''
    Wrapper for smtplib; sends emails via Gmail
    Requires permission from Gmail to allow less-secure login methods on the account; other SMTP email servers are also possible
    '''
    import smtplib

    gmail_user = em_user
    gmail_pwd = em_pwd
    FROM = gmail_user
    TO = recipients
    SUBJECT = subject 
    TEXT = body + " \n This message generated by Todd's Janky Monitoring System"

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()
        server.login(gmail_user, gmail_pwd)
        server.sendmail(FROM, TO, message)
        server.close()
        print ('successfully sent the mail')
    except:
        print ("failed to send mail")

In [2]:
def query_mysql(local_bound_host='127.0.0.1', local_bound_port=3306, user='read_only', pw='', db='public', query=[]):
    '''
    Connects to mysql through a tunnel that already opened, then executes queries in the query list
    
    '''
    import pymysql, math

    conn = pymysql.connect(host=local_bound_host, port=local_bound_port, user=user, passwd=pw, db=db)
    q_results=[]
    
    for q in query:
        q_cursor = conn.cursor()
        q_cursor.execute(q)
        q_result.append(q_cursor)
        q_cursor.close()
    
    
    #Compare results
    per_min=[]
    for qr in q_results
        for i in qr:
            per_min.append(i[2])

    combined = list(map(float,per_min))
    time_diff = combined[1]/combined[0]-1
    
    #check to see if lats 10 minutes are within a certain ratio of last hour. Currently looking for 25% deviations
    if abs(time_diff) > 0.25:
        email_text=  '''
                      Last 10 min checkin numbers deviating from hourly mean by {3}.
                      Last 10 min checkins per minute: {0}.
                      Last hour checkins per minute: {1}.
                      For detailed differences, please run MySQL queries: 
                      {2}

                       '''.format(combined[0],combined[1],' and '.join(query),time_diff)
        #send email if things are bad
        send_email(body=email_text, subject= 'Alert: Last 10 Min Checkins Deviation')

    print(combined)


    conn.close()


In [3]:
#TODO: Open Tunnel....
## Defines the tunnel opening parametesr
#!pip install pymysql
def open_tunnel_and_query_mysql(ssh_url='testserver.com',ssh_username='user_name',ssh_password='optional_pw', ssh_pkey_loc='', remote_bind_url='', remote_bind_port=3306, local_bind_url='127.0.0.1', local_bind_port=3306):
    '''
    This is a wrapper around paramiko and sshtunnel that opens a tunnel to a host at ssh_url, which can be host string or a tuple of (host, port)
    The key difference in this is that the SSH key
    
    Please modify the variables as needed    
    '''
    import paramiko
    from sshtunnel import SSHTunnelForwarder
    #ssh_url can
    server = SSHTunnelForwarder(
        ssh_url,
        ssh_username,
        ssh_password,
        ssh_pkey=paramiko.RSAKey.from_private_key(open(ssh_pkey_loc)),

        remote_bind_address=(remote_bind_url, remote_bind_port),
        local_bind_address=(local_bind_url, local_bind_port)
    )


    try:
        server.start()
        print('Tunnel success on port: ')
        print(server.local_bind_port)
    except:
        print('Error on tunnel to server')
        
    # query mySQL server that has been connected locally; please modify as needed; build or import query list as needed.
    
    try:
        query_mysql(local_bound_host=local_bind_url, local_bound_port=local_bind_port, user='read_only', pw='', db='public', query=[])
    except:
        print('Error on MySQL query execution')
    server.close()


In [ ]:
#Schedule and keep alive. THIS IS THE "MAIN" and it starts everything else!

#!pip install schedule

import schedule 
import time

def job():
    open_tunnel_and_query_mysql()
    print('Ran job successfully')

def okay_email():
    send_email(subject = 'System Okay', body = "System Still Active",em_user='xxx', em_pwd='xxx', recipients=['test@example.com'])

schedule.every(15).minutes.do(job)
schedule.every(1).day.at("10:00").do(okay_email)
#schedule.every().hour.do(job)
#schedule.every().monday.do(job)
#schedule.every().wednesday.at("13:15").do(job)

while True:
    schedule.run_pending()
    time.sleep(10)
    
